# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import OrderedDict
import train

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=32)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [5]:
list(cat_to_name.items())[:4]

[('21', 'fire lily'),
 ('3', 'canterbury bells'),
 ('45', 'bolero deep blue'),
 ('1', 'pink primrose')]

In [6]:
num_classes = len(cat_to_name)
num_classes

102

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [7]:
# TODO: Build and train your network

# use VGG with 16 layers as suggested in https://arxiv.org/pdf/1409.1556.pdf
model = models.vgg16(weights='DEFAULT')

# switch off gradient computation for features
for param in model.parameters():
    param.requires_grad = False

# determine the size of the input layer for the classifier part
classifier_in_features_num = model.classifier[0].in_features

# create custom classifier for our problem
classifier = torch.nn.Sequential(OrderedDict([
                          ('fc1', torch.nn.Linear(in_features=classifier_in_features_num, out_features=4096, bias=True)),
                          ('relu1', torch.nn.ReLU(inplace=True)),
                          ('dropout1', torch.nn.Dropout(p=0.5, inplace=False)),
                          ('fc2', torch.nn.Linear(in_features=4096, out_features=4096, bias=True)),
                          ('relu2', torch.nn.ReLU(inplace=True)),
                          ('dropout2', torch.nn.Dropout(p=0.5, inplace=False)),
                          ('fc3', torch.nn.Linear(in_features=4096, out_features=num_classes, bias=True)),
                          ('output', torch.nn.LogSoftmax(dim=1))
                          ]))

# replace the default classifier with our custom classifier
model.classifier = classifier

In [8]:
model.classifier

Sequential(
  (fc1): Linear(in_features=25088, out_features=4096, bias=True)
  (relu1): ReLU(inplace=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (relu2): ReLU(inplace=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=4096, out_features=102, bias=True)
  (output): LogSoftmax(dim=1)
)

In [9]:
torch.cuda.is_available()

True

In [10]:
def train(device, model, train_loader, test_loader, criterion, optimizer, epochs=5, print_every=10):
    model.to(device)
    steps = 0
    running_loss = 0
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            steps += 1
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)
            
            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            if steps % print_every == 0:
                valid_loss = 0
                accuracy = 0
                model.eval()
                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        logps = model.forward(inputs)
                        batch_loss = criterion(logps, labels)
                        
                        valid_loss += batch_loss.item()
                        
                        # Calculate accuracy
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                        
                print(f"Epoch {epoch+1}/{epochs}, "
                    f"Train loss: {running_loss/print_every:.3f}, "
                    f"Test loss: {valid_loss/len(test_loader):.3f} "
                    f"Accuracy: {accuracy/len(test_loader):.3f}")
                running_loss = 0
                model.train()

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.NLLLoss()

for learning_rate in [0.0005, 0.0008, 0.001, 0.0012, 0.0015]:
    print(f'Learning rate: {learning_rate}')
    # reset classifier weights
    model.classifier.reset_parameters()
    optimizer = torch.optim.Adam(model.classifier.parameters(), lr=learning_rate)
    train_losses, test_losses, accuracies = train(device, model, train_loader, valid_loader, criterion, optimizer, epochs=1, print_every=20)

Learning rate: 0.0005
Epoch 1/1, Train loss: 4.771, Test loss: 4.221 Accuracy: 0.159
Epoch 1/1, Train loss: 4.168, Test loss: 3.571 Accuracy: 0.268
Epoch 1/1, Train loss: 3.660, Test loss: 2.815 Accuracy: 0.345
Epoch 1/1, Train loss: 3.105, Test loss: 2.471 Accuracy: 0.381
Epoch 1/1, Train loss: 2.781, Test loss: 1.897 Accuracy: 0.509
Epoch 1/1, Train loss: 2.605, Test loss: 1.679 Accuracy: 0.563
Epoch 1/1, Train loss: 2.391, Test loss: 1.593 Accuracy: 0.572
Epoch 1/1, Train loss: 2.174, Test loss: 1.390 Accuracy: 0.616
Epoch 1/1, Train loss: 2.089, Test loss: 1.388 Accuracy: 0.626
Epoch 1/1, Train loss: 1.992, Test loss: 1.260 Accuracy: 0.665
Learning rate: 0.001
Epoch 1/1, Train loss: 2.514, Test loss: 1.548 Accuracy: 0.625
Epoch 1/1, Train loss: 2.414, Test loss: 1.361 Accuracy: 0.623
Epoch 1/1, Train loss: 2.278, Test loss: 1.176 Accuracy: 0.677
Epoch 1/1, Train loss: 2.389, Test loss: 1.263 Accuracy: 0.658
Epoch 1/1, Train loss: 2.398, Test loss: 1.308 Accuracy: 0.623
Epoch 1/1, T

best learning rate for VGG16, NLLLoss and Adam is 0.001, because it leads to good validation accuracy at a comparitively high learning rate

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.NLLLoss()

best_learning_rate = 0.0008
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=best_learning_rate)

train(device, model, train_loader, valid_loader, criterion, optimizer, epochs=10, print_every=32)

Epoch 1/10, Train loss: 4.764, Test loss: 3.903 Accuracy: 0.201
Epoch 1/10, Train loss: 3.732, Test loss: 3.002 Accuracy: 0.299
Epoch 1/10, Train loss: 3.232, Test loss: 2.383 Accuracy: 0.437
Epoch 1/10, Train loss: 2.858, Test loss: 1.882 Accuracy: 0.496
Epoch 1/10, Train loss: 2.732, Test loss: 1.653 Accuracy: 0.580
Epoch 1/10, Train loss: 2.707, Test loss: 1.546 Accuracy: 0.574
Epoch 2/10, Train loss: 2.507, Test loss: 1.547 Accuracy: 0.573
Epoch 2/10, Train loss: 2.516, Test loss: 1.411 Accuracy: 0.614
Epoch 2/10, Train loss: 2.554, Test loss: 1.305 Accuracy: 0.648
Epoch 2/10, Train loss: 2.482, Test loss: 1.299 Accuracy: 0.645
Epoch 2/10, Train loss: 2.481, Test loss: 1.344 Accuracy: 0.626
Epoch 2/10, Train loss: 2.369, Test loss: 1.306 Accuracy: 0.624
Epoch 3/10, Train loss: 2.389, Test loss: 1.314 Accuracy: 0.658
Epoch 3/10, Train loss: 2.348, Test loss: 1.096 Accuracy: 0.709
Epoch 3/10, Train loss: 2.427, Test loss: 1.135 Accuracy: 0.691
Epoch 3/10, Train loss: 2.344, Test loss

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [15]:
def test(device, model, test_loader, criterion):
    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            
            test_loss += batch_loss.item()
            
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
    print(f"Test loss: {test_loss/len(test_loader):.3f} "
        f"Accuracy: {accuracy/len(test_loader):.3f}")

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test(device, model, test_loader, criterion)

Test loss: 1.032 Accuracy: 0.774


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [24]:
model.class_to_idx = train_data.class_to_idx
model.class_to_idx

{'1': 0,
 '10': 1,
 '100': 2,
 '101': 3,
 '102': 4,
 '11': 5,
 '12': 6,
 '13': 7,
 '14': 8,
 '15': 9,
 '16': 10,
 '17': 11,
 '18': 12,
 '19': 13,
 '2': 14,
 '20': 15,
 '21': 16,
 '22': 17,
 '23': 18,
 '24': 19,
 '25': 20,
 '26': 21,
 '27': 22,
 '28': 23,
 '29': 24,
 '3': 25,
 '30': 26,
 '31': 27,
 '32': 28,
 '33': 29,
 '34': 30,
 '35': 31,
 '36': 32,
 '37': 33,
 '38': 34,
 '39': 35,
 '4': 36,
 '40': 37,
 '41': 38,
 '42': 39,
 '43': 40,
 '44': 41,
 '45': 42,
 '46': 43,
 '47': 44,
 '48': 45,
 '49': 46,
 '5': 47,
 '50': 48,
 '51': 49,
 '52': 50,
 '53': 51,
 '54': 52,
 '55': 53,
 '56': 54,
 '57': 55,
 '58': 56,
 '59': 57,
 '6': 58,
 '60': 59,
 '61': 60,
 '62': 61,
 '63': 62,
 '64': 63,
 '65': 64,
 '66': 65,
 '67': 66,
 '68': 67,
 '69': 68,
 '7': 69,
 '70': 70,
 '71': 71,
 '72': 72,
 '73': 73,
 '74': 74,
 '75': 75,
 '76': 76,
 '77': 77,
 '78': 78,
 '79': 79,
 '8': 80,
 '80': 81,
 '81': 82,
 '82': 83,
 '83': 84,
 '84': 85,
 '85': 86,
 '86': 87,
 '87': 88,
 '88': 89,
 '89': 90,
 '9': 91,
 '90

In [23]:
model.classifier.state_dict()

OrderedDict([('fc1.weight',
              tensor([[-8.6216e-03,  1.1311e-03,  4.5222e-03,  ...,  3.4496e-03,
                        3.6735e-05, -5.0760e-03],
                      [ 4.1055e-03,  5.0968e-03,  7.9693e-03,  ..., -1.0752e-02,
                       -3.8354e-03, -2.7297e-03],
                      [-4.1799e-03, -7.8749e-03, -7.7072e-03,  ..., -1.0663e-02,
                       -3.6037e-04, -3.7683e-03],
                      ...,
                      [-2.7223e-03, -1.4739e-03, -4.4981e-03,  ..., -5.7961e-04,
                        3.1358e-03, -7.7630e-03],
                      [ 5.8307e-03, -2.2884e-02,  5.4619e-02,  ...,  2.0751e-02,
                        2.3087e-02, -2.5759e-02],
                      [ 4.8443e-02, -2.6774e-02, -2.9270e-02,  ...,  1.4618e-03,
                        1.7528e-02,  1.5851e-02]], device='cuda:0')),
             ('fc1.bias',
              tensor([-0.0047, -0.0042, -0.0094,  ..., -0.0013, -0.0143,  0.0072],
                     device='c

In [29]:
# TODO: Save the checkpoint 
EPOCH = 10
PATH = "checkpoint.pth"

torch.save({
            'epoch': EPOCH,
            'class_to_index': model.class_to_idx,
            'classifier_state_dict': model.classifier.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, PATH)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [10]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_model(device, model, optimizer, PATH):
    checkpoint = torch.load(PATH, map_location=device)
    model.classifier.load_state_dict(checkpoint['classifier_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    model.class_to_idx = checkpoint['class_to_index']
    return epoch

In [17]:
PATH = "checkpoint.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.0008)
criterion = torch.nn.NLLLoss()

epoch = load_model(device, model, optimizer, PATH)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes